### Progress-1

In [3]:
import time

from selenium.webdriver.ie.webdriver import WebDriver
from tqdm import tqdm

# 模擬一個長時間運行的任務
for i in tqdm(range(100), desc="Loading..."):
    time.sleep(0.1)  # 模擬任務

Loading...: 100%|██████████| 100/100 [00:10<00:00,  9.54it/s]


### Progress-2 (infiniti loading)

In [5]:
import itertools
import sys
import time

def always_loading(desc="Loading...", delay=0.1):
    spinner = itertools.cycle(['-', '\\', '|', '/'])
    while True:
        sys.stdout.write(f'\r{desc} {next(spinner)}')
        sys.stdout.flush()
        time.sleep(delay)

try:
    always_loading()
except KeyboardInterrupt:
    sys.stdout.write('\rDone!     \n')
    sys.stdout.flush()

Done!      /


### Progress-3 (countdown)

In [7]:
import time
from tqdm import tqdm
from datetime import datetime, timedelta

def countdown_to(target_time):
    now = datetime.now()
    target = datetime.strptime(target_time, "%Y-%m-%d %H:%M:%S")
    delta = (target - now).total_seconds()

    if delta <= 0:
        print(f"Target date is expired => {target_time}")
        return

    for i in tqdm(range(int(delta)), desc="AWaiting time for auto-apply execution", unit="s", bar_format="{desc}: {remaining}"):
        time.sleep(1)

# 設定目標時間
target_time = "2025-09-05 00:05:00"
countdown_to(target_time)

AWaiting time for auto-apply execution: 00:57


KeyboardInterrupt: 

In [ ]:
import logging
def init_log():
    import logging

    # 設定日誌格式
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    
    # 建立日誌記錄器
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    
    # 建立文件處理器
    file_handler = logging.FileHandler('example.log')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    
    # 建立控制台處理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    
    # 將處理器添加到日誌記錄器
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    return logger

In [ ]:
import os
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def find_chromedriver():
    # 方法1：使用 shutil.which()
    path = shutil.which('chromedriver')
    if path:
        return path

    # 方法2：檢查常見路徑
    common_paths = [
        '/usr/bin/chromedriver',
        '/usr/local/bin/chromedriver',
        os.path.expanduser('~/chromedriver')
    ]

    for common_path in common_paths:
        if os.path.exists(common_path):
            return common_path

    # 方法3：遍歷 PATH
    for path in os.environ.get('PATH', '').split(os.pathsep):
        potential_path = os.path.join(path, 'chromedriver')
        if os.path.exists(potential_path):
            return potential_path

    return None

# 使用找到的 ChromeDriver 路徑
chromedriver_path = find_chromedriver()
if chromedriver_path:
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)
else:
    print("ChromeDriver not found")

### Get Chromedriver install path

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def get_chromedriver_path():
    try:
        # 初始化 Chrome WebDriver
        service = Service()
        driver = webdriver.Chrome(service=service)
        # 取得 ChromeDriver 的執行檔路徑
        chromedriver_path = service.path
        driver.quit()
        return chromedriver_path
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

chromedriver_path = get_chromedriver_path()
if chromedriver_path:
    print(f"ChromeDriver path: {chromedriver_path}")
else:
    print("Failed to get ChromeDriver path.")

### connect to remote webdriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# service = Service('/Users/shihxuancheng/.cache/selenium/chromedriver/mac-arm64/131.0.6778.85/chromedriver')
# service.start()
options = Options()
driver = webdriver.Remote('http://127.0.0.1:57922',options=options)
# driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
driver.quit()

### 取得NTP標準時間

In [ ]:
import ntplib
from time import ctime

def get_ntp_time():
    client = ntplib.NTPClient()
    # response = client.request('pool.ntp.org')
    response = client.request('clear')
    return ctime(response.tx_time)
print("NTP Time:", get_ntp_time())

### 計算本機到特定url的傳輸時間 (TCP連接建立時間加上初始HTTP請求時間)

In [ ]:
import requests
import time
import statistics
from tqdm import tqdm

def test_connection_time(url, num_requests=10):
    """
    測試本機與指定URL間的連接時間，不等待完整回應

    參數:
        url (str): 要測試的URL
        num_requests (int): 請求次數

    返回:
        dict: 包含平均、最小、最大和所有連接時間的字典
    """
    connection_times = []

    print(f"測試URL連接: {url}")
    print(f"執行 {num_requests} 次連接測試...\n")

    # 使用tqdm顯示進度條
    for i in tqdm(range(num_requests), desc="測試進度"):
        start_time = time.time()
        try:
            # 使用 stream=True 和 timeout 參數
            # 只測量建立連接的時間，不下載完整回應
            session = requests.Session()
            response = session.get(
                url,
                stream=True,    # 不立即下載回應內容
                timeout=(5, 5)  # (連接超時, 讀取超時) - 短讀取超時
            )
            # 立即關閉連接
            response.close()

        except requests.exceptions.RequestException as e:
            print(f"連接失敗: {e}")
            continue

        end_time = time.time()
        connection_time = (end_time - start_time) * 1000  # 轉換為毫秒
        connection_times.append(connection_time)

        # 避免頻繁請求導致伺服器拒絕
        time.sleep(0.1)

    if not connection_times:
        return {"error": "所有連接都失敗了"}

    # 計算統計數據
    avg_time = statistics.mean(connection_times)
    min_time = min(connection_times)
    max_time = max(connection_times)
    median_time = statistics.median(connection_times)

    return {
        "平均連接時間(ms)": round(avg_time, 2),
        "最小連接時間(ms)": round(min_time, 2),
        "最大連接時間(ms)": round(max_time, 2),
        "中位數連接時間(ms)": round(median_time, 2),
        "所有連接時間": [round(t, 2) for t in connection_times]
    }

# target utl
target_url = 'https://docs.google.com/forms/d/1hHHrf19cWw0Nn8C0RIgOwXwhcUQJSuNpqMoQCERuQVI'
results = test_connection_time(target_url, num_requests=3)

print("\n測試結果:")
for key, value in results.items():
    if key != "所有回應時間":
        print(f"{key}: {value}")

print("\n所有回應時間(ms):")
print(results["所有連接時間"])



### Convert date field of chromedriver log

In [ ]:
import re
import datetime

# 讀取 chromedriver 日誌文件
with open('./chromedrver-2025-09-04.log', 'r', encoding='utf-8') as file:
    log_content = file.read()

# 正則表達式匹配時間戳記格式
# 匹配包含小數點的時間戳記格式
pattern = r'\[(\d+\.\d+)\]'


def timestamp_to_readable(match):
    timestamp = float(match.group(1))
    # 將秒級時間戳轉換為日期時間格式
    readable_time = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return f'[{readable_time}]'


# 替換所有時間戳記
readable_log = re.sub(pattern, timestamp_to_readable, log_content)

# 將可閱讀的日誌寫入新文件
with open('./chromedriver_readable.log', 'w') as file:
    file.write(readable_log)
    print(f"converted file: {file.name}")

print("轉換完成！")
